In [1]:
with open('input') as f:
    compressed_text = f.read().replace('\n', '')

In [2]:
def decompress(txt):
    bracket_open = False
    bracket_text = ""
    a = b = 0
    output = ""
    repeat_txt = ""
    for char in txt:
        if a > 0:
            repeat_txt += char
            a -= 1
            if a == 0:
                output += repeat_txt * b
                repeat_txt = ""
        elif bracket_open:
            if char == ')':
                bracket_open = False
                a, b = [int(n) for n in bracket_text.split('x')]
                bracket_text = ""
                skip = a
            else:
                bracket_text += char
        elif char == '(':
            bracket_open = True
        else:
            output += char
    return output

In [3]:
assert (a := decompress("ADVENT")) == "ADVENT", a
assert (a := decompress("A(1x5)BC")) == "ABBBBBC", a
assert (a := decompress("(3x3)XYZ")) == "XYZXYZXYZ", a
assert (a := decompress("A(2x2)BCD(2x2)EFG")) == "ABCBCDEFEFG", a
assert (a := decompress("(6x1)(1x3)A")) == "(1x3)A", a
assert (a := decompress("X(8x2)(3x3)ABCY")) == "X(3x3)ABC(3x3)ABCY", a

In [4]:
print("Part 1:")
print(len(decompress(compressed_text)))

Part 1:
152851


In [5]:
class Chunk:
    @classmethod
    def from_text(self, text):
        if text.startswith('('):
            return Marker(text)
        else:
            return Text(text)

In [6]:
class Marker(Chunk):
    def __init__(self, text):
        self.text = text
        
    def __len__(self):
        return len(self.text)
    
    def __repr__(self):
        return self.text if self.is_active else 'NONE'
    
    @property
    def a(self):
        return int(self.text[1:-1].split('x')[0])
    
    @property
    def b(self):
        return int(self.text[1:-1].split('x')[1])
    
    @property
    def is_active(self):
        return self.text is not None
    
    def get_n_chars(self, n):
        text = self.text
        self.text = None
        return text

In [7]:
class Text:
    def __init__(self, text):
        self.text_len = len(text)
        
    def __len__(self):
        return self.text_len
    
    def __repr__(self):
        return str(len(self))
    
    @property
    def is_active(self):
        return len(self) > 0
    
    def get_n_chars(self, n):
        if n > len(self):
            text_len = len(self)
            self.text_len = 0
            return text_len * 'A'
        else:
            self.text_len -= n
            return n * 'A'

In [8]:
def make_chunks(text):
    chunks = []
    chunk = ''
    bracket_open = False
    for char in text:
        if bracket_open and char == ')':
            chunk += char
            chunks.append(Chunk.from_text(chunk))
            chunk = ''
            bracket_open = False
        elif char == '(':
            bracket_open = True
            if chunk:
                chunks.append(Chunk.from_text(chunk))
            chunk = char
        else:
            chunk += char
    if chunk:
        chunks.append(Chunk.from_text(chunk))
    return chunks

In [9]:
def get_next_n_chars(chunks, n):
    chars = ''
    for chunk in chunks:
        chars += chunk.get_n_chars(n - len(chars))
        if len(chars) == n:
            return chars

In [10]:
def partial_decompress(chunks):
    new_chunks = []
    for i in range(len(chunks)):
        chunk = chunks[i]
        if chunk.is_active:
            if type(chunk) == Text:
                new_chunks.append(chunk)
            else:
                repeat_chars = get_next_n_chars(chunks[i+1:], n=chunk.a)
                for _ in range(chunk.b):
                    new_chunks.extend(make_chunks(repeat_chars))
    return new_chunks

In [11]:
def get_decompress_length(chunks):
    while not all(type(chunk) == Text for chunk in chunks):
        chunks = partial_decompress(chunks)
    return sum(len(chunk) for chunk in chunks)

In [12]:
assert (a := get_decompress_length(make_chunks("(3x3)XYZ"))) == 9, a
assert (a := get_decompress_length(make_chunks("X(8x2)(3x3)ABCY"))) == 20, a
assert (a := get_decompress_length(make_chunks("(27x12)(20x12)(13x14)(7x10)(1x12)A"))) == 241920, a
assert (a := get_decompress_length(make_chunks("(25x3)(3x3)ABC(2x3)XY(5x2)PQRSTX(18x9)(3x2)TWO(5x7)SEVEN"))) == 445, a

In [13]:
get_decompress_length(make_chunks(compressed_text))

KeyboardInterrupt: 